In [ ]:
import numpy as np
import xarray as xr
import scipy.stats as st
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import requests
import os,errno
import sys
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import datetime as dt
import pandas as pd
import time
import urllib.request
import metpy.calc as mpcalc
import salem
import scipy.optimize as opt
import warnings
import geopy.distance
import matplotlib as mpl
from scipy.spatial import ConvexHull

from scipy.ndimage.measurements import label
from scipy.ndimage import binary_dilation

warnings.filterwarnings('ignore')

dir_data='/Users/lexihenny/'
dir3='/Volumes/My Passport/'
dir2='/Volumes/Extreme Pro 1/'
dir4='/Volumes/Extreme Pro/'

In [ ]:
yrs=np.arange(1980,2020,1)
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    #ds=xr.open_dataset(dir2+'era5.ar.tempestlr'+str(year)+'.nc')
    #ds=xr.open_dataset(dir2+'era5.ar.mundhenkv3.'+str(year)+'.nc')
    ds=xr.open_dataset(dir2+'era5.ar.reid500.'+str(year)+'.nc')
    #ds=xr.open_dataset(dir2+'merra2.ar.cndl.'+str(year)+'.nc')
    #ds=xr.open_dataset(dir2+'merra2.ar.acv2.'+str(year)+'.nc')
    #ds=xr.open_dataset(dir2+'merra2.ar.lorav2.'+str(year)+'.nc')
    #ds=xr.open_dataset(dir2+'jra55.ar.gwv2.'+str(year)+'.nc')
    ar=ds['ar'].values
    
    ds=xr.open_dataset(dir2+'era_5_ivtinterp_'+str(year)+'.nc')
    ivt=ds['ivt'].values
    
    #ds=xr.open_dataset(dir2+'merra2.ivt.3hr.model.'+str(year)+'.nc')
    #ivt=ds['ivt'][0::2,:,:].values
    
    #ds=xr.open_dataset(dir4+'jra55_ivt_'+str(year)+'.nc')
    #ivt=ds['ivt']
    #ivt=ivt.interp_like(ar).values
    #ar=ar.values
    
    ivt_max=np.where(ar==1,ivt,0)
    
    
    fig=plt.figure(figsize=(20,10))
    ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
    ax.pcolormesh(ds.lon,ds.lat,ivt[818,:,:],vmin=0,vmax=1000,cmap=plt.cm.seismic)
    ax.coastlines(resolution='10m')
    plt.show()
    
    fig=plt.figure(figsize=(20,10))
    ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
    ax.pcolormesh(ds.lon,ds.lat,ivt_max[816,:,:],vmin=0,vmax=1000,cmap=plt.cm.seismic)
    ax.coastlines(resolution='10m')
    plt.show()
    
    fig=plt.figure(figsize=(20,10))
    ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
    ax.pcolormesh(ds.lon,ds.lat,ar[816,:,:],vmin=0,vmax=1,cmap=plt.cm.seismic)
    ax.coastlines(resolution='10m')
    plt.show()
    sys.exit()
    
    ivt_max=np.amax(ivt_max,axis=0)
    
    if i==0:
        max_concat=ivt_max[np.newaxis,:,:]
    else:
        max_concat=np.concatenate((max_concat,ivt_max[np.newaxis,:,:]),axis=0)
        
    print(max_concat.shape)

In [ ]:
if 1==1:
    year=2019
    print(year)
    #ds=xr.open_dataset(dir2+'era5.ar.tempestlr'+str(year)+'.nc')
    ds=xr.open_dataset(dir2+'era5.ar.mundhenkv3.'+str(2019)+'.nc')
    ar=ds['ar'].values
    
    ds=xr.open_dataset(dir2+'era_5_ivtinterp_'+str(year)+'.nc')
    ivt=ds['ivt'].values

    ivt_max=np.where(ar==1,ivt,0)
    ivt_max=np.amax(ivt_max,axis=0)

    max_concat=np.concatenate((max_concat,ivt_max[np.newaxis,:,:]),axis=0)
        
    print(max_concat.shape)

In [ ]:
all_max=np.amax(max_concat,axis=0)

In [ ]:
ds=xr.open_dataset(dir2+'jra55.ar.gwv2.'+str(year)+'.nc')
yrs=np.arange(1980,2020,1)
dk=xr.Dataset()
dk['max_concat']=(('time','lat','lon'),max_concat)
dk.coords['time']=yrs
dk.coords['lat']=ds.lat
dk.coords['lon']=ds.lon
try:
    os.remove(dir_data+'ar_max_climo_jra55gw.nc')
except OSError:
    pass
dk.to_netcdf(dir_data+'ar_max_climo_jra55gw.nc',mode='w',format='NETCDF4')

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import ListedColormap


ds=xr.open_dataset(dir_data+'ar_max_climo_jra55gw.nc')
max_concat=ds['max_concat'].values
all_max=np.amax(max_concat,axis=0)

colors1=plt.cm.viridis(np.linspace(0,1,128))
colors2=LinearSegmentedColormap.from_list('new_cmap',(colors1[-1],'orange'))
colors2=colors2(np.linspace(0,1,32))
colors3=LinearSegmentedColormap.from_list('test',('orange','chocolate'))
colors3=colors3(np.linspace(0,1,32))
colors4=LinearSegmentedColormap.from_list('test',('chocolate','saddlebrown'))
colors4=colors4(np.linspace(0,1,32))


colors5=np.vstack((colors1,colors2,colors3,colors4))
cmap_new=ListedColormap(colors5)

fig=plt.figure(figsize=(22,10))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
#cax=ax.pcolormesh(ds.lon,ds.lat,all_max,vmin=0,vmax=3450,cmap=cmap_new)
cax=ax.contourf(ds.lon,ds.lat,all_max,levels=np.arange(100,3700,200),extend='max',cmap=cmap_new)
cbar=plt.colorbar(cax,pad=0,fraction=0.046)
cbar.ax.tick_params(labelsize=20)
cbar.set_label('kg m$^{-1}$ s$^{-1}$',fontsize=22)
ax.coastlines(resolution='10m')
#ax.set_title(r'$\bf{ERA5}$'+' '+r'$\bf{TempestLR}$',fontsize=36,pad=5)
#ax.set_title(r'$\bf{ERA5}$'+' '+r'$\bf{Reid500}$',fontsize=36,pad=5)
#ax.set_title(r'$\bf{ERA5}$'+' '+r'$\bf{Mundhenk\_v3}$',fontsize=36,pad=5)
#ax.set_title(r'$\bf{MERRA-2}$'+' '+r'$\bf{Lora\_v2}$',fontsize=36,pad=5)
#ax.set_title(r'$\bf{MERRA-2}$'+' '+r'$\bf{ClimateNet\_DL}$',fontsize=36,pad=5)
#ax.set_title(r'$\bf{MERRA-2}$'+' '+r'$\bf{ARCONNECT\_v2}$',fontsize=36,pad=5)
ax.set_title(r'$\bf{JRA-55}$'+' '+r'$\bf{GuanWaliser}$',fontsize=36,pad=5)

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='k',alpha=0.5,linestyle='--')
#ax.add_feature(cfeature.STATES.with_scale('10m'),alpha=0.3)
#ax.add_feature(cfeature.LAKES.with_scale('50m'))
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':20,'color':'k'}
g1.ylabel_style={'size':20,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False


plt.show()

In [ ]:
fig.savefig(dir_data+'ar_max_climo_7.png',dpi=150)

In [ ]:
#yrs=np.arange(1980,2020,1)
yrs=np.arange(2020,2024,1)
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    
    ds=xr.open_dataset(dir4+'era_5_ivtinterp_'+str(year)+'.nc')
    ivt=ds['ivt'].values
    ivt_max=np.amax(ivt,axis=0)
    
    indices=np.argmax(ivt,axis=0)
    
    #fig=plt.figure(figsize=(20,10))
    #ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
    #ax.pcolormesh(ds.lon,ds.lat,indices,vmin=0,vmax=1460,cmap=plt.cm.seismic)
    #ax.coastlines(resolution='10m')
    #plt.show()
    #sys.exit()
    
    #if i==0:
    if 1==0:
        max_concat=ivt_max[np.newaxis,:,:]
        time_concat=indices[np.newaxis,:,:]
    else:
        max_concat=np.concatenate((max_concat,ivt_max[np.newaxis,:,:]),axis=0)
        time_concat=np.concatenate((time_concat,indices[np.newaxis,:,:]),axis=0)
        
    #print(max_concat.shape)

In [ ]:
all_max=np.amax(max_concat,axis=0)
times_final=np.full_like(all_max,np.nan)
indices_all_max=np.argmax(max_concat,axis=0)
for j in range(len(all_max[:,0])):
    for k in range(len(all_max[0,:])):
        index=indices_all_max[j,k]
        times_final[j,k]=time_concat[index,j,k]

In [ ]:
print(max_concat.shape)

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import ListedColormap

colors1=plt.cm.viridis(np.linspace(0,1,128))
colors2=LinearSegmentedColormap.from_list('new_cmap',(colors1[-1],'orange'))
colors2=colors2(np.linspace(0,1,32))
colors3=LinearSegmentedColormap.from_list('test',('orange','chocolate'))
colors3=colors3(np.linspace(0,1,32))
colors4=LinearSegmentedColormap.from_list('test',('chocolate','saddlebrown'))
colors4=colors4(np.linspace(0,1,32))


colors5=np.vstack((colors1,colors2,colors3,colors4))
cmap_new=ListedColormap(colors5)

fig=plt.figure(figsize=(22,10))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
#cax=ax.pcolormesh(ds.lon,ds.lat,all_max,vmin=0,vmax=3450,cmap=cmap_new)
cax=ax.contourf(ds.lon,ds.lat,all_max,levels=np.arange(100,3700,200),extend='max',cmap=cmap_new)
cbar=plt.colorbar(cax,pad=0,fraction=0.046)
cbar.ax.tick_params(labelsize=20)
cbar.set_label('kg m$^{-1}$ s$^{-1}$',fontsize=22)
ax.coastlines(resolution='10m')
ax.set_title(r'$\bf{ERA5}$'+' '+r'$\bf{1980-2023}$'+' '+r'$\bf{max}$'+' '+r'$\bf{IVT}$',fontsize=36,pad=5)

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='k',alpha=0.5,linestyle='--')
#ax.add_feature(cfeature.STATES.with_scale('10m'),alpha=0.3)
#ax.add_feature(cfeature.LAKES.with_scale('50m'))
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':20,'color':'k'}
g1.ylabel_style={'size':20,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False


plt.show()

In [ ]:
fig.savefig(dir_data+'ivt_max_climo_1.png',dpi=200)

In [ ]:
colors1=plt.cm.hsv_r(np.linspace(0.25,1,128))
colors2=plt.cm.hsv_r(np.linspace(0,0.25,43))
colors=np.vstack((colors1,colors2))
cmap_new=ListedColormap(colors)

fig=plt.figure(figsize=(22,10))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
#cax=ax.pcolormesh(ds.lon,ds.lat,all_max,vmin=0,vmax=3450,cmap=cmap_new)
#cax=ax.contourf(ds.lon,ds.lat,times_final,levels=np.arange(0,1501,100),cmap=cmap_new)
cax=ax.pcolormesh(ds.lon,ds.lat,times_final,vmin=0,vmax=1464,cmap=cmap_new)

cbar=plt.colorbar(cax,pad=0,fraction=0.046,ticks=[60,184,296,420,540,664,784,908,1032,1152,1276,1396])
cbar.ax.set_yticklabels(['January','February','March','April','May','June','July','August','September','October','November','December'])
cbar.ax.tick_params(labelsize=20)
ax.coastlines(resolution='10m')
ax.set_title(r'$\bf{ERA5}$'+' '+r'$\bf{1980-2023}$'+' '+r'$\bf{max}$'+' '+r'$\bf{IVT}$'+' '+r'$\bf{dates}$',fontsize=36,pad=5)

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='k',alpha=0.5,linestyle='--')
#ax.add_feature(cfeature.STATES.with_scale('10m'),alpha=0.3)
#ax.add_feature(cfeature.LAKES.with_scale('50m'))
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':20,'color':'k'}
g1.ylabel_style={'size':20,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False


plt.show()

In [ ]:
fig.savefig(dir_data+'ivt_max_climo_2.png',dpi=200)